In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pydatafabric.vault_utils import get_secrets

oracle_info = get_secrets(mount_point="datafabric",path="oracle/datafabric/oracle-db")
aws_info = get_secrets(mount_point="datafabric",path="aws/credentials/datafabric")

In [3]:
from pydatafabric.ye import get_spark

spark = get_spark(extra_jars="gs://emart-datafabric-resources/jars/ojdbc8.jar")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/30 12:51:42 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/09/30 12:51:42 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/09/30 12:51:42 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/09/30 12:51:42 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [4]:
spark.conf.set("fs.s3a.fast.upload.buffer", "disk")
spark.conf.set("fs.s3a.buffer.dir", "/tmp")
spark.conf.set("fs.s3a.access.key", aws_info["aws_access_key_id"])
spark.conf.set("fs.s3a.secret.key", aws_info["aws_secret_access_key"])
spark.conf.set("spark.sql.debug.maxToStringFields", 2000)

In [5]:
df = spark.read.format("jdbc") \
    .option("url", oracle_info['jdbc_url']) \
    .option("user", oracle_info['user']) \
    .option("password", oracle_info['password']) \
    .option("driver", "oracle.jdbc.driver.OracleDriver") \
    .option("dbtable", "(SELECT * FROM C##DPC.GTPSD_EP_PAY_RESULT_A) a") \
    .load()

In [6]:
df.show()

+------------+--------+------+-------+----------+--------------+--------------------+--------------+--------------------+----------+---------------+-------------------+---------+-------------------+---------+
|POS_STR_CODE|BIZ_DATE|POS_NO|TRAN_NO|TRAN_TCODE|TRAN_TYPE_CODE|            BAR_CODE|    TIME_STAMP|        MCH_ORDER_NO|PAY_RESULT|            MSG|            CRTN_DT|  CRTN_ID|             CHG_DT|   CHG_ID|
+------------+--------+------+-------+----------+--------------+--------------------+--------------+--------------------+----------+---------------+-------------------+---------+-------------------+---------+
|        4700|20220310|  1032|   8563|        00|            00|27182371100613712...|20220310140522| 2022031014050904185|     00000|             OK|2022-03-10 23:43:07|TRNROUTER|2022-03-10 23:43:07|TRNROUTER|
|        4700|20220310|  1032|   8564|        00|            00|27182400652754379...|20220310140729| 2022031014071904186|     00000|             OK|2022-03-10 23:43

In [7]:
s3_path = "s3a://emart-datafabric-dev/oracle-db/"

df.write.mode("overwrite").parquet(s3_path)

In [8]:
!hdfs dfs -Dhadoop.security.credential.provider.path=jceks:///datafabric/credentials/aws.jceks -ls {s3_path}

2022-09-30 12:54:18,981 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-09-30 12:54:19,032 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-09-30 12:54:19,032 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
Found 2 items
-rw-rw-rw-   1 datafabric datafabric          0 2022-09-30 12:53 s3a://emart-datafabric-dev/oracle-db/_SUCCESS
-rw-rw-rw-   1 datafabric datafabric     366518 2022-09-30 12:53 s3a://emart-datafabric-dev/oracle-db/part-00000-859d4936-8fb3-4735-8510-ca3444b84961-c000.snappy.parquet
2022-09-30 12:54:20,665 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-09-30 12:54:20,665 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-09-30 12:54:20,665 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


In [12]:
spark.stop()